Basic CNN

In [1]:

%matplotlib inline

if not os.path.isdir('models'):
    os.mkdir('models')
    
print('TensorFlow version:', tf.__version__)
print('Is using GPU?', tf.test.is_gpu_available())

TensorFlow version: 2.12.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Is using GPU? False


In [1]:
import cv2
import keras
import tensorflow as tf
import os
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import keras.utils as image
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('harrasment_as_csv.csv')
train.head()

,image,class
0,images/harassment_yes\Screenshot 2023-07-22 01...,1
1,images/harassment_yes\Screenshot 2023-07-22 01...,1
2,images/harassment_yes\Screenshot 2023-07-22 01...,1
3,images/harassment_yes\Screenshot 2023-07-22 01...,1
4,images/harassment_yes\Screenshot 2023-07-22 01...,1


In [5]:
# creating an empty list
train_image = []

# for loop to read and store frames
for i in tqdm(range(train.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img(train['image'][i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255.0
    # appending the image to the train_image list
    train_image.append(img)
    
# converting the list to numpy array
x = np.array(train_image)

# shape of the array
x.shape

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:07<00:00, 136.43it/s]


(1000, 224, 224, 3)

In [6]:
# separating the target
y = train['class']

# creating the training and validation set
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42, test_size=0.2, stratify = y)

In [7]:
# creating dummies of target variable for train and validation set
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [8]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Dropout, Flatten, Input, Dense

def create_model():
    
    def add_conv_block(model, num_filters):
        
        model.add(Conv2D(num_filters, 3, activation='relu', padding='same'))
        model.add(BatchNormalization())
        model.add(Conv2D(num_filters, 3, activation='relu', padding='valid'))
        model.add(MaxPooling2D(pool_size=2))
        model.add(Dropout(0.2))

        return model
    
    model = tf.keras.models.Sequential()
    model.add(Input(shape=(224, 224, 3)))
    
    model = add_conv_block(model, 32)
    model = add_conv_block(model, 64)
    model = add_conv_block(model, 128)
    model = add_conv_block(model, 256)

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 224, 224, 32)      128       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 222, 222, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 111, 111, 32)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 111, 111, 64)      1

In [9]:
%%time

h = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=200, batch_size=128
    )

Epoch 1/200
7/7 [==============================] - 81s 11s/step - loss: 19.8910 - accuracy: 0.4950 - val_loss: 2.0888 - val_accuracy: 0.5000
Epoch 2/200
7/7 [==============================] - 76s 11s/step - loss: 5.5332 - accuracy: 0.5275 - val_loss: 0.7179 - val_accuracy: 0.4350
Epoch 3/200
7/7 [==============================] - 77s 11s/step - loss: 2.3302 - accuracy: 0.4863 - val_loss: 1.6707 - val_accuracy: 0.5000
Epoch 4/200
7/7 [==============================] - 77s 11s/step - loss: 1.0927 - accuracy: 0.4963 - val_loss: 0.9155 - val_accuracy: 0.4950
Epoch 5/200
7/7 [==============================] - 74s 10s/step - loss: 0.8393 - accuracy: 0.4650 - val_loss: 0.6908 - val_accuracy: 0.4950
Epoch 6/200
7/7 [==============================] - 74s 10s/step - loss: 0.7188 - accuracy: 0.4950 - val_loss: 0.6926 - val_accuracy: 0.5000
Epoch 7/200
7/7 [==============================] - 76s 11s/step - loss: 0.6956 - accuracy: 0.5088 - val_loss: 0.6896 - val_accuracy: 0.5100
Epoch 8/200
7/7 [==

In [10]:
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")

7/7 [==============================] - 3s 359ms/step - loss: 1.4641 - accuracy: 0.8400
Test Accuracy: 83.99999737739563


In [11]:
#saving our model
model.save('harrasment_cnn_model.h5')

C:\Users\sudha\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
